## Example loading of CytoFM weights into a ViT
- Code is based on that found in the iBOT Github: https://github.com/bytedance/ibot
- Need to have pulled iBOT code to run this notebook!

In [9]:
import sys
import os

import torch

sys.path.append("../../bytedance_ibot/ibot/")

from models import VisionTransformer, vit_tiny, vit_small, vit_base, vit_large
from models import SwinTransformer, swin_tiny, swin_small, swin_base
    
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
torch.cuda.is_available()

True

In [3]:
# path to model weights
model_weights = 'cytofm_weights.pth'

In [4]:
# load weights
ck = torch.load(model_weights, map_location=torch.device('cpu'))

In [52]:
# load teacher weights into vit_base (configured in ibot code)
output_dict = dict(state_dict=dict())
has_backbone = False
for key, value in ck['teacher'].items():
    #print(key)
    if key.startswith('backbone'):
        output_dict['state_dict'][key[9:]] = value
        has_backbone = True
    elif key.startswith('module.backbone'):
        output_dict['state_dict'][key[16:]] = value
        has_backbone = True
vit = vit_base(patch_size=16).cuda()
vit.load_state_dict(output_dict['state_dict'], strict=True)
vit.eval();
for p in vit.parameters():
    p.requires_grad = False

In [53]:
# check model loaded correctly and runs on an image

img = torch.randn([1, 3, 224, 224])

with torch.no_grad():
    features = vit(img.cuda())
features.shape

torch.Size([1, 768])